In [1]:
import pandas as pd
import numpy as np
import snowflake.connector
from collections import defaultdict
import json
import re
import requests
import sys
import random
import requests
import tabulate
from datetime import datetime
import schedule
import time

In [2]:
#login to snowflake db
con = snowflake.connector.connect(user='vishal.kumar@scale.com',
                                 account='pxa65918',
                                 authenticator='externalbrowser',
                                 warehouse='COMPUTE_WH',
                                 database='SCALE_CRAWLER',
                                 role='GENERAL_RO')

cs = con.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [10]:
#Get work logs
sql = f'''
with fb as (
  SELECT
    qe._id as ID,
    qe.user_email :: string uemail,
    qe.metadata: action action,
    qe.metadata: auditField auditField,
    qe.metadata: domain domain,
    qe.metadata: eventCreatedAt eventCreatedAt,
    qe.metadata: fieldAdditions fieldAdditions,
    qe.metadata: fieldBefore fieldBefore,
    qe.metadata: fieldCurrent fieldCurrent,
    qe.metadata: fieldLength fieldLength,
    qe.metadata: ruleCreated ruleCreated,
    qe.metadata: fieldRemovals fieldRemovals,
    qe.metadata: flagsAvailable flagsAvailable,
    qe.metadata: hintsAvailable hintsAvailable,
--  qe.metadata: pvids pvids,
    qe.metadata: totalMillisElapsed totalMillisElapsed,
    qe.event_type,
    qe.event_at,
    cast (qe.event_at as DATE) AS EventDate,
    qe.metadata: auditLevel :: string auditLevel,
    pv.value:: string as pvid
  FROM
    PUBLIC.QAEVENTS qe,
    lateral flatten(input => qe.metadata: pvids) pv
--    lateral flatten(input => pv.value, outer => true) pvd
  WHERE
    CAST(qe.event_at AS DATE) > '2022-09-20'
    AND qe.metadata :domain is not NULL
    AND qe.user_email :: string like '%vishal%'
   -- AND qe.event_type = 'submit_description_speed_audit' --  AND qe.metadata: auditLevel !='QA'
    -- AND qe.user_email LIKE '%tele%'
)
select
  fb.ID,
  fb.uemail,
  fb.action,
  fb.auditField,
  fb.domain,
  fb.eventCreatedAt,
  fb.fieldAdditions,
  fb.fieldBefore,
  fb.fieldCurrent,
  fb.fieldLength,
  fb.fieldRemovals,
  fb.ruleCreated,
  fb.flagsAvailable,
  fb.hintsAvailable,
  fb.pvid,
  fb.totalMillisElapsed,
  fb.event_type,
  fb.event_at,
  fb.EventDate
from
  fb
where 
fb.EventDate > '2022-09-13'
AND fb.domain like '%albionfit%'
'''
cs.execute(sql)
tasklogdf = cs.fetch_pandas_all()

In [11]:
#tasklogdf.to_csv('tasks.csv')
#tasklogdf.loc[tasklogdf['PVID'] == 'zonenaturals.com!29037423394896']
tasklogdf.to_csv('dump3.csv')

In [4]:
tasklogdf.groupby(['DOMAIN'])['FIELDREMOVALS'].count()

DOMAIN
"albionfit.com"    5957
Name: FIELDREMOVALS, dtype: int64

In [5]:
#tasklogdf.astype({'FIELDLENGTH': 'int32'}).dtypes
tasklogdf['FIELDLENGTH'] = pd.to_numeric(tasklogdf['FIELDLENGTH'])

In [6]:
domdf = tasklogdf.groupby(['DOMAIN'],['FIELDBEFORE'],['FIELDREMOVALS'],['UEMAIL'])['ID'].count()
#domdf = pd.DataFrame([domdf])

TypeError: unhashable type: 'list'

In [ ]:
domdf

In [ ]:
#taskdf1 = tasklogdf.groupby(['PVID'])['ID'].nunique()
#taskdf = tasklogdf.groupby(['PVID'])['FIELDCOUNT'].count()
#taskdf = tasklogdf.groupby(['PVID'])['FIELDLENGTH'].agg(lambda x: pd.to_numeric(x, errors='coerce').sum())
#taskdf = tasklogdf.groupby(['PVID'])['ID'].count()
#taskdf = pd.pivot_table(tasklogdf, index=['PVID'],values=['ID','FIELDLENGTH'],aggfunc=np.sum)
taskdf = pd.pivot_table(
   tasklogdf,
   index=['DOMAIN'],
   aggfunc={'FIELDLENGTH': np.sum, 'ID': len}
).rename(columns={'ID': 'COUNT'})

In [ ]:
taskdf['CHANGELENGTH'] = taskdf['FIELDLENGTH']/taskdf['COUNT']

In [ ]:
#taskdf['CHANGED'] = (taskdf['CHANGELENGTH']).is_integer()

In [ ]:
#tasklogdf['AUDITLEVEL'].unique()

In [ ]:
taskdf['FRACTION'] = taskdf['CHANGELENGTH'].astype(str).str.split('.').str[1].astype(int)

In [ ]:
taskdf

In [ ]:
changedpv = taskdf.loc[taskdf['FRACTION'] != 0]

In [ ]:
changedpv

In [ ]:
findf = changedpv.merge(tasklogdf, left_on='PVID', right_on='PVID', how='inner', indicator=True)

In [ ]:
findf

In [ ]:
tasklogdf.loc[tasklogdf['PVID'] == 'albionfit.com!15385906118759']

In [ ]:
findf['LEVEL'] = np.where(findf['UEMAIL'].str.contains("contractors"), 'QA', np.where(findf['UEMAIL'].str.contains("telework"), 'TW', 'ScaleHQ'))

In [ ]:
findf.loc[findf['LEVEL'] == 'TW']

In [ ]:
findf.to_csv('auditsdesc2.csv')

In [ ]:
findf = findf.drop('AUDITFIELD', axis=1)

In [ ]:
findf